In [1]:
#import the needed package
import requests
import pandas as pd
import numpy as np
from bokeh.plotting import figure, output_file, show, output_notebook
from bokeh.models import NumeralTickFormatter
from bokeh.io import show
from bokeh.layouts import column
from bokeh.models import ColumnDataSource, CustomJS, Select
from bokeh.plotting import figure

In [2]:
# Call the PV Watts to get the data (prediction)
nrel_long_tilt = []

# We choose the tilt at the below degree, and the Fairbanks lon & lat are (64.82,-147.87)
tilts = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90]
for i in range(len(tilts)):
    list_parameters = {"formt": 'JSON', "api_key": "spJFj2l5ghY5jwk7dNfVYs3JHbpR6BOGHQNO8Y9Z", "system_capacity": 4, "module_type": 0, "losses": 14.08,
              "array_type": 0, "tilt": tilts[i], "azimuth": 180, "lat": 64.82, "lon": -147.87, "dataset": 'tmy2'}
    json_response = requests.get("https://developer.nrel.gov/api/pvwatts/v6", params = list_parameters).json()
    new_dataframe = pd.DataFrame(data = json_response['outputs'])
    nrel_long_tilt.append(new_dataframe)

In [7]:
annual_production = []
for i in range(len(tilts)):
    annual_production.append(nrel_long_tilt[i]['ac_annual'][2])

d = {'Tilts':tilts,'Annual_production':annual_production}
df = pd.DataFrame(d)

#Then find out the max production raw
max_tilt = int(df[['Annual_production']].idxmax().values)

#Then calculate the other tilts' lose compared with the max annual production
lose = []
for index, row in df.iterrows():
    tilt_loss = 1- row['Annual_production']/df['Annual_production'][max_tilt]
    lose.append(tilt_loss)

df['loss']=lose

output_file("Fairbanks_tilts_loss.html")

p = figure(x_axis_label='Tilts', y_axis_label='loss (%)',plot_width=400, plot_height=250)

# add a line renderer
p.line(tilts, df['loss'], line_width=2)
p.xaxis.ticker = [10,20,30,40,50,60,70,80,90]
p.yaxis.formatter = NumeralTickFormatter(format='0 %')
p.title.text = "Annual Production loss of different tilts"
p.title.align = "center"
p.title.text_color = "olive"
p.title.text_font = "times"
p.title.text_font_style = "italic"
p.title.text_font_size = '12pt'

show(p)